# Paths

In [5]:
# Specify paths
images_folder = r'C:\Users\User\Documents\dataset\pfas\34759_final_project_rect\seq_02\image_02\data'
labels_file = r'C:\Users\User\Documents\dataset\pfas\34759_final_project_rect\seq_02\labels.txt'

# Functions

In [6]:
def get_frame_data(frame_id, labels_file):
    """
    Extracts and returns track_ids and bounding boxes for a given frame from the labels file.

    Parameters:
    frame_id (int): The ID of the frame to extract information for.
    labels_file (str): Path to the labels.txt file containing bounding box information.

    Returns:
    tuple: A tuple containing:
        - frame_ids (list): List of track_ids present in the frame.
        - frame_locations (dict): Dictionary with track_id as key and bounding box [x1, y1, x2, y2] as value.
    """
    # Load labels from the file
    with open(labels_file, 'r') as file:
        labels = file.readlines()

    # Filter labels for the specified frame_id
    frame_ids = []
    frame_locations = {}

    for line in labels:
        columns = line.strip().split()
        if int(columns[0]) == frame_id:
            track_id_raw = columns[1]
            try:
                track_id = int(track_id_raw)
            except ValueError:
                track_id = track_id_raw  # Keep as string if it cannot be converted

            x1, y1, x2, y2 = map(float, columns[6:10])  # Extract top-left and bottom-right corners

            # Append the track_id and bounding box information
            frame_ids.append(track_id)
            frame_locations[track_id] = [x1, y1, x2, y2]

    return frame_ids, frame_locations


In [9]:
# Test get_frame_data function
frame_id = 0
frame_ids, frame_locations = get_frame_data(frame_id, labels_file)
print(f'Frame {frame_id}:')
print(f'frame_ids: {frame_ids}')
print(f'frame_locations: {frame_locations}')

Frame 0:
frame_ids: [0, 1, 2, 3, 4, 5, 6, 7, 22, 23, 24, 25, 26]
frame_locations: {0: [1096.141118, 185.415106, 1223.0, 236.828782], 1: [1031.968582, 183.785008, 1157.636703, 232.560471], 2: [952.605364, 181.930859, 1068.611605, 234.494197], 3: [602.556774, 172.407984, 636.777103, 202.727588], 4: [324.388946, 170.363766, 423.513983, 316.413307], 5: [733.172819, 157.632371, 783.506153, 281.860744], 6: [782.480296, 166.494327, 817.146964, 275.918322], 7: [436.728458, 161.218615, 476.395123, 254.699107], 22: [275.187184, 156.595011, 307.187184, 230.127117], 23: [314.211664, 161.032118, 340.211663, 227.50554], 24: [209.727761, 154.940942, 237.727761, 220.178333], 25: [244.404816, 154.153304, 265.071484, 218.579195], 26: [271.163454, 159.400648, 298.926237, 218.175444]}


In [ ]:
def track_states(current_frame_ids, previous_frame_ids):
    """
    Classifies object states into missing, tracked, and new based on IDs in the current and previous frames.

    Parameters:
    current_frame_ids (list): List of track_ids detected in the current frame.
    previous_frame_ids (list): List of track_ids detected in the previous frame.

    Returns:
    tuple: A tuple containing:
        - missing_ids (list): IDs present in previous_frame_ids but not in current_frame_ids.
        - tracked_ids (list): IDs present in both current_frame_ids and previous_frame_ids.
        - new_ids (list): IDs present in current_frame_ids but not in previous_frame_ids.
    """
    # Determine missing IDs (present in previous_frame_ids but not in current_frame_ids)
    missing_ids = [id_ for id_ in previous_frame_ids if id_ not in current_frame_ids]

    # Determine tracked IDs (present in both current_frame_ids and previous_frame_ids)
    tracked_ids = [id_ for id_ in current_frame_ids if id_ in previous_frame_ids]

    # Determine new IDs (present in current_frame_ids but not in previous_frame_ids)
    new_ids = [id_ for id_ in current_frame_ids if id_ not in previous_frame_ids]

    return missing_ids, tracked_ids, new_ids

In [11]:
# Test track_states function
frame_id = 0
previous_frame_ids, _ = get_frame_data(frame_id, labels_file)

frame_id = 1
current_frame_ids, _ = get_frame_data(frame_id, labels_file)

missing_ids, tracked_ids, new_ids = track_states(current_frame_ids, previous_frame_ids)

# Print the results
print(f'Frame {frame_id}:')
print(f'missing_ids: {missing_ids}')
print(f'tracked_ids: {tracked_ids}')
print(f'new_ids: {new_ids}')

Frame 1:
missing_ids: []
tracked_ids: [0, 1, 2, 3, 4, 5, 6, 7, 22, 23, 24, 25, 26]
new_ids: []


# The filter

In [12]:
# Step 1: Variable Initialization
missing_ids = []                     # IDs of objects missing in the current frame
tracked_ids = []                     # IDs of objects being tracked
new_ids = []                         # IDs of new detections in the current frame
locations_dict = {}                  # Bounding boxes for each track_id {track_id: [x1, y1, x2, y2]}
previous_frame_ids = []              # IDs detected in the previous frame
previous_frame_locations = {}        # Bounding boxes from the previous frame
frame_counter = 1                    # Counter for the current frame
reassociation_map = {}               # Maps new_ids to missing_ids
kalman_filters = {}                  # Kalman filter instances for each track_id
missing_counts = {}                  # Counts how long each missing_id has been missing

# Constants
max_distance = 50                    # Maximum distance for matching (tunable)
max_missing_frames = 30               # Maximum frames an object can be missing

In [15]:
# Step 2: Retrieve Frame Data

if frame_counter == 1:
    # Retrieve data for frame 1 (current frame)
    current_frame_ids, current_frame_locations = get_frame_data(frame_counter, labels_file)
    # Retrieve data for frame 0 (previous frame)
    previous_frame_ids, previous_frame_locations = get_frame_data(0, labels_file)
else:
    # Retrieve data for the current frame
    current_frame_ids, current_frame_locations = get_frame_data(frame_counter, labels_file)
    # Use data from the previous iteration for comparison
    previous_frame_ids = previous_frame_ids.copy()
    previous_frame_locations = previous_frame_locations.copy()

# Debugging output (optional)
print("Frame Counter:", frame_counter)
print("Current Frame IDs:", current_frame_ids)
print("Current Frame Locations:", current_frame_locations)
print("Previous Frame IDs:", previous_frame_ids)
print("Previous Frame Locations:", previous_frame_locations)


Frame Counter: 1
Current Frame IDs: [0, 1, 2, 3, 4, 5, 6, 7, 22, 23, 24, 25, 26]
Current Frame Locations: {0: [1096.127247, 185.40352, 1223.0, 236.816016], 1: [1031.972644, 183.786106, 1157.631279, 232.561457], 2: [952.60759, 181.929214, 1068.612769, 234.492229], 3: [602.556357, 172.409683, 636.775583, 202.729325], 4: [302.69041, 170.414513, 411.413539, 325.511953], 5: [740.540847, 157.261852, 793.723504, 282.769239], 6: [786.093082, 166.840239, 827.77389, 277.196724], 7: [442.839462, 161.030879, 485.71239, 255.206338], 22: [276.028463, 156.615028, 307.06085, 230.650286], 23: [312.702509, 161.319478, 340.576901, 228.23184], 24: [210.263112, 154.571909, 238.381584, 220.247145], 25: [243.428067, 154.253899, 263.703316, 219.091944], 26: [272.150418, 159.366907, 299.66969, 218.464906]}
Previous Frame IDs: [0, 1, 2, 3, 4, 5, 6, 7, 22, 23, 24, 25, 26]
Previous Frame Locations: {0: [1096.141118, 185.415106, 1223.0, 236.828782], 1: [1031.968582, 183.785008, 1157.636703, 232.560471], 2: [952.60

In [19]:
# Fake data for testing
current_frame_ids = [1, 2, 3]
current_frame_locations = {1: [10, 10, 20, 20], 2: [30, 30, 40, 40], 3: [50, 50, 60, 60]}

previous_frame_ids = [1, 2, 4]
previous_frame_locations = {1: [10, 10, 20, 20], 2: [30, 30, 40, 40], 4: [70, 70, 80, 80]}

In [20]:
# Step 3: Determine States
missing_ids, tracked_ids, new_ids = track_states(current_frame_ids, previous_frame_ids)

# Debugging output (optional)
print("Missing IDs:", missing_ids)
print("Tracked IDs:", tracked_ids)
print("New IDs:", new_ids)

Missing IDs: [4]
Tracked IDs: [1, 2]
New IDs: [3]


In [22]:
# Step 4: Process Missing IDs
def predict_missing_locations(missing_ids, kalman_filters):
    """
    Predicts the locations of missing objects using their Kalman filters.

    Parameters:
    missing_ids (list): IDs of objects missing in the current frame.
    kalman_filters (dict): Dictionary of Kalman filter instances keyed by track_id.

    Returns:
    dict: Predicted locations of missing objects as {track_id: [x1, y1, x2, y2]}.
    """
    missing_ids_prediction = {}

    for missing_id in missing_ids:
        if missing_id in kalman_filters:
            # Predict the next state using the Kalman filter
            kalman_filters[missing_id].predict()
            # Retrieve the predicted location
            predicted_location = kalman_filters[missing_id].get_predicted_location()
            # Store the prediction
            missing_ids_prediction[missing_id] = predicted_location
        else:
            print(f"Warning: No Kalman filter for missing_id {missing_id}")

    return missing_ids_prediction

In [23]:
# Step 4: Predict locations for missing IDs
missing_ids_prediction = predict_missing_locations(missing_ids, kalman_filters)

# Debugging output (optional)
print("Predicted locations for Missing IDs:", missing_ids_prediction)


Predicted locations for Missing IDs: {}
